<a href="https://colab.research.google.com/github/Himanshu0702/Machine-Learning/blob/main/Mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
pip install image


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [55]:
pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
import numpy as np
import cv2
import PIL
import PIL.Image as Image
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [57]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", input_shape=IMAGE_SHAPE+(3,))
])

In [58]:
IMAGE_SHAPE+(3,)

(224, 224, 3)

In [59]:
data_dir = '/content/drive/MyDrive/Skin_Disease'

In [60]:
import pathlib 
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('/content/drive/MyDrive/Skin_Disease')

In [61]:
list(data_dir.glob('*/*.jpg'))[:4]

[PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/atypical-nevi-dermoscopy-98.jpg'),
 PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/congenital-nevus-27.jpg'),
 PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/congenital-nevus-32.jpg'),
 PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/atypical-nevi-26.jpg')]

In [62]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3592


In [63]:
Melanoma = list(data_dir.glob('Melanoma Skin Cancer Nevi and Moles/*'))
Melanoma[:5]

[PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/atypical-nevi-dermoscopy-98.jpg'),
 PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/congenital-nevus-27.jpg'),
 PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/congenital-nevus-32.jpg'),
 PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/atypical-nevi-26.jpg'),
 PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/congenital-nevus-55.jpg')]

In [64]:
diseases_images_dict = {
    'Melanoma' : list(data_dir.glob('Melanoma Skin Cancer Nevi and Moles/*')),
    'Eczema' : list(data_dir.glob('Eczema Photos/*')),
    'Dermatitis' : list(data_dir.glob('Atopic Dermatitis Photos/*')),
    'Psoriasis' : list(data_dir.glob('Psoriasis pictures Lichen Planus and related diseases/*')),
}

In [65]:
diseases_images_dict['Melanoma']

[PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/atypical-nevi-dermoscopy-98.jpg'),
 PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/congenital-nevus-27.jpg'),
 PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/congenital-nevus-32.jpg'),
 PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/atypical-nevi-26.jpg'),
 PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/congenital-nevus-55.jpg'),
 PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/atypical-nevi-44.jpg'),
 PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/atypical-nevi-trunk-10.jpg'),
 PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/atypical-nevi-dermoscopy-79.jpg'),
 PosixPath('/content/drive/MyDrive/Skin_Disease/Melanoma Skin Cancer Nevi and Moles/atypical-nevi-4

In [66]:
diseases_labels_dict = {
    'Melanoma' : 0,
    'Eczema' : 1,
    'Dermatitis': 2,
    'Psoriasis': 3,
}

In [67]:
img = cv2.imread(str(diseases_images_dict['Melanoma'][1]))
img.shape

(464, 720, 3)

In [68]:
cv2.resize(img,IMAGE_SHAPE).shape

(224, 224, 3)

In [71]:
X, y = [], []

for disease_name, images in diseases_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        X.append(resized_img)
        y.append(diseases_labels_dict[disease_name])

In [74]:
X = np.array(X)
y = np.array(y)

In [75]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [76]:
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [ ]:
plt.axis('off')
plt.imshow(X[0])

In [ ]:
plt.axis('off')
plt.imshow(X[1])

In [81]:
predicted = classifier.predict(np.array([X[0],X[1],X[2]]))
predicted = np.argmax(predicted, axis=1)
predicted

1/1 [==============================] - 0s 75ms/step


array([795, 795, 795])

In [86]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224,224,3), trainable=False)


In [87]:
num_of_disease = 4

model = tf.keras.Sequential([
    pretrained_model_without_top_layer,
    tf.keras.layers.Dense(num_of_disease)
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 4)                 5124      
                                                                 
Total params: 2,263,108
Trainable params: 5,124
Non-trainable params: 2,257,984
_________________________________________________________________


In [88]:
model.compile(
    optimizer = "adam",
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
85/85 [==============================] - 55s 616ms/step - loss: 1.1840 - acc: 0.4870
Epoch 2/5
85/85 [==============================] - 51s 597ms/step - loss: 0.9119 - acc: 0.6114
Epoch 3/5
85/85 [==============================] - 49s 578ms/step - loss: 0.8176 - acc: 0.6578
Epoch 4/5
85/85 [==============================] - 49s 580ms/step - loss: 0.7575 - acc: 0.6860
Epoch 5/5
85/85 [==============================] - 49s 577ms/step - loss: 0.7109 - acc: 0.7101


In [89]:
model.evaluate(X_test_scaled, y_test)

29/29 [==============================] - 17s 569ms/step - loss: 0.8546 - acc: 0.6336


[0.854591965675354, 0.6336302757263184]